In [9]:
!pip install pandas
import pandas as pd
import t3_utils as t3
from ast import literal_eval

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# change the file path here
file_path_base = r"/home/dani/Dropbox/JKU/Semester-7/Multimedia_Search_and_Retrieval/Excercises/MMSR-Group-C/3/datasets/"
file_path_info = file_path_base + "id_information_mmsr.tsv" 
file_path_word2vec = file_path_base + "id_lyrics_word2vec_mmsr.tsv"
file_path_tfidf = file_path_base + "id_lyrics_tf-idf_mmsr.tsv"
file_path_bert = file_path_base + "id_lyrics_bert_mmsr.tsv"

file_path_genre = file_path_base + "id_genres_mmsr.tsv"
file_path_musicnn = file_path_base + "id_musicnn_mmsr.tsv"
file_path_mfcc_bow = file_path_base + "id_mfcc_bow_mmsr.tsv"
file_path_ivec_256 = file_path_base + "id_ivec256_mmsr.tsv"
file_path_logfluc = file_path_base + "id_blf_logfluc_mmsr.tsv"

file_path_incp = file_path_base  + "id_incp_mmsr.tsv"
file_path_vgg19 = file_path_base + "id_vgg19_mmsr.tsv"
file_path_resnet = file_path_base + "id_resnet_mmsr.tsv"
file_path_url = file_path_base + "id_url_mmsr.tsv"

df_info = pd.read_table(file_path_info)

# text
df_word2vec = pd.read_table(file_path_word2vec)
df_tfidf = pd.read_table(file_path_tfidf)
df_bert = pd.read_table(file_path_bert)

# audio
df_genre = pd.read_table(file_path_genre)
df_musicnn = pd.read_table(file_path_musicnn)
df_mfcc_bow = pd.read_table(file_path_mfcc_bow)
df_ivec_256 = pd.read_table(file_path_ivec_256)
df_logfluc = pd.read_table(file_path_logfluc)

# video
df_incp = pd.read_table(file_path_incp)
df_vgg19 = pd.read_table(file_path_vgg19)
df_resnet = pd.read_table(file_path_resnet)
df_url = pd.read_table(file_path_url)

df_genre["genre"] = df_genre["genre"].apply(literal_eval) # convert string into array

# todo: prepare early fusion feature
# normalizes and joins both features
#df_normalized_bert = t3.normalize_frame(df_bert)
#df_normalized_mfcc_bow = t3.normalize_frame(df_mfcc_bow)
#df_early_fusion = pd.merge(df_normalized_bert, df_normalized_mfcc_bow, on="id", how="outer")

In [11]:
from youtubesearchpython import VideosSearch

def get_youtube_url(row):
    song_artist = row["artist"]
    song_title = row["song"]

    query = f"{song_title} {song_artist}"
    videos_search = VideosSearch(query, limit=1)

    results = videos_search.result()

    if results['result']:
        video_url = results['result'][0]['link']
        print(row["id"])
        return video_url
    else:
        print(f"No results found for {song_title} by {song_artist}")
        return ''
    

def calc_songs(row, df_info: pd.DataFrame, df_feature: pd.DataFrame, sim_func, filter: []):
    id = row["id"]
    song_artist = row["artist"]
    song_title = row["song"]
    
    filter = [] if filter is None else filter

    df_retr_songs = t3.song_retrieval(df_info, df_feature, song_title, song_artist, 10, sim_func=sim_func, filter=filter)
   
    return df_retr_songs


In [12]:
import os # Import df_info with youtube links

file_path = "df_info_and_links.json"
if os.path.exists(file_path):
    df_info = pd.read_json(file_path)
else:
    df_info["youtube_link"] = df_info.apply(get_youtube_url, axis=1)
    df_info.to_json(orient='records', indent=2, path_or_buf=f'{file_path}')

In [13]:
# insert genres for each song
df_info = pd.merge(df_info, df_genre, on = "id")
df_info

,id,artist,song,album_name,youtube_link,genre
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human,https://www.youtube.com/watch?v=v2T40qPAO-0,"[rock, christian rock]"
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition),https://www.youtube.com/watch?v=22YBfS3hLrg,"[hip hop, rap, grindcore, death metal]"
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones,https://www.youtube.com/watch?v=f7dx1GgO7CI,"[rock, pop punk]"
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002),https://www.youtube.com/watch?v=mNUFRFlwDPw,"[trance, techno, progressive trance]"
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te,https://www.youtube.com/watch?v=gl17x1L_8Kc,"[pop, italian pop, latin, europop, ambient, po..."
...,...,...,...,...,...,...
10089,zyzILCQvVeUFIINi,Crowded House,When You Come,Temple Of Low Men,https://www.youtube.com/watch?v=LUP3XOa3yRs,"[rock, pop, indie rock, folk rock, new wave, m..."
10090,zzgS4ZqyswamEWNj,Britney Spears,My Only Wish (This Year),Platinum Christmas,https://www.youtube.com/watch?v=_MzumcY3lpk,"[pop, rock, teen pop, soundtrack, singer songw..."
10091,zzoFYDMlqU1X2zz1,Thundercat,DUI,Drunk,https://www.youtube.com/watch?v=O0kS88zAzh8,"[jazz, jazz fusion]"
10092,zzpkRCGA5ud8q4mv,Otis Redding,Rock Me Baby,Otis Blue,https://www.youtube.com/watch?v=rKpI3o-1CSM,"[soul, blues, r b, blues rock, southern soul, ..."


In [14]:
rs_info = [("word2vec euc-sim", df_word2vec, t3.euc_sim),
                ("tfidf cos-sim", df_tfidf, t3.cos_sim), ("bert cos-sim", df_bert, t3.cos_sim),
                ("mfcc_bow cos-sim", df_mfcc_bow, t3.cos_sim), ("musicnn cos-sim", df_musicnn, t3.cos_sim),
                ("ivec_256 cos-sim", df_ivec_256, t3.cos_sim), ("logfluc cos-sim", df_logfluc, t3.cos_sim),
               ("resnet cos-sim", df_resnet, t3.cos_sim)
           #, ("early fusion cos-sim", df_early_fusion, t3.cos_sim)
           # todo: consider early fusion and stuff?
           ]

In [15]:
# This is Danis json version
df_info = df_info.head(10)
df_dani = df_info.copy()

# if an error occurs: please delete the song "03Oc9WeMEmyLLQbj" ind information.csv
for index, feature_set, sim_func in rs_info:
    df_dani[f'res_songs_{index}'] = df_info.apply(calc_songs, axis=1, args=(df_info, feature_set, sim_func, ["id"]))
    print(f"\nFinished {index} -------------------")

df_dani.to_json(orient='records', indent=2, path_or_buf=f'data-dani.json')
df_dani

01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 

Finished word2vec euc-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 

Finished tfidf cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 

Finished bert cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 

Finished mfcc_bow cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9

/home/dani/Dropbox/JKU/Semester-7/Multimedia_Search_and_Retrieval/Excercises/MMSR-Group-C/3/t3_utils.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(query,target)/(norm(query)*norm(target))


01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 

Finished logfluc cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 

Finished resnet cos-sim -------------------


,id,artist,song,album_name,youtube_link,genre,res_songs_word2vec euc-sim,res_songs_tfidf cos-sim,res_songs_bert cos-sim,res_songs_mfcc_bow cos-sim,res_songs_musicnn cos-sim,res_songs_ivec_256 cos-sim,res_songs_logfluc cos-sim,res_songs_resnet cos-sim
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human,https://www.youtube.com/watch?v=v2T40qPAO-0,"[rock, christian rock]",id 7 04xUDjAYC14jsHyH 2 01r...,id 1 01gyRHLquwXDlhkO 4 02Z...,id 5 04OjszRi9rC5BlHC 1 01g...,id 3 02RGE9FNH65RtMS7 1 01g...,id 7 04xUDjAYC14jsHyH 2 01r...,id 7 04xUDjAYC14jsHyH 5 04O...,id 5 04OjszRi9rC5BlHC 1 01g...,id 3 02RGE9FNH65RtMS7 9 06L...
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition),https://www.youtube.com/watch?v=22YBfS3hLrg,"[hip hop, rap, grindcore, death metal]",id 2 01rMxQv6vhyE1oQX 7 04x...,id 0 01Yfj2T3YTwJ1Yfy 6 04i...,id 5 04OjszRi9rC5BlHC 0 01Y...,id 4 02ZnlCGZEbkfCDxo 6 04i...,id 2 01rMxQv6vhyE1oQX 4 02Z...,id 8 06HvNTU9M9lnH71I 0 01Y...,id 0 01Yfj2T3YTwJ1Yfy 8 06H...,id 9 06L9OJ5nRqKnO2q9 2 01r...
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones,https://www.youtube.com/watch?v=f7dx1GgO7CI,"[rock, pop punk]",id 7 04xUDjAYC14jsHyH 8 06H...,id 9 06L9OJ5nRqKnO2q9 6 04i...,id 5 04OjszRi9rC5BlHC 7 04x...,id 4 02ZnlCGZEbkfCDxo 5 04O...,id 4 02ZnlCGZEbkfCDxo 6 04i...,id 7 04xUDjAYC14jsHyH 4 02Z...,id 8 06HvNTU9M9lnH71I 6 04i...,id 1 01gyRHLquwXDlhkO 0 01Y...
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002),https://www.youtube.com/watch?v=mNUFRFlwDPw,"[trance, techno, progressive trance]",id 7 04xUDjAYC14jsHyH 0 01Y...,id 9 06L9OJ5nRqKnO2q9 8 06H...,id 7 04xUDjAYC14jsHyH 9 06L...,id 0 01Yfj2T3YTwJ1Yfy 1 01g...,id 1 01gyRHLquwXDlhkO 4 02Z...,id 4 02ZnlCGZEbkfCDxo 5 04O...,id 0 01Yfj2T3YTwJ1Yfy 5 04O...,id 6 04iitW3ffa0mhpx3 4 02Z...
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te,https://www.youtube.com/watch?v=gl17x1L_8Kc,"[pop, italian pop, latin, europop, ambient, po...",id 7 04xUDjAYC14jsHyH 8 06H...,id 3 02RGE9FNH65RtMS7 7 04x...,id 7 04xUDjAYC14jsHyH 9 06L...,id 2 01rMxQv6vhyE1oQX 1 01g...,id 6 04iitW3ffa0mhpx3 2 01r...,id 8 06HvNTU9M9lnH71I 5 04O...,id 8 06HvNTU9M9lnH71I 7 04x...,id 3 02RGE9FNH65RtMS7 6 04i...
5,04OjszRi9rC5BlHC,Grizzly Bear,Knife,Yellow House,https://www.youtube.com/watch?v=PVdGjSSarIQ,"[experimental, folk, lo fi, freak folk, indie ...",id 6 04iitW3ffa0mhpx3 1 01g...,id 9 06L9OJ5nRqKnO2q9 2 01r...,id 9 06L9OJ5nRqKnO2q9 7 04x...,id 1 01gyRHLquwXDlhkO 8 06H...,id 4 02ZnlCGZEbkfCDxo 9 06L...,id 6 04iitW3ffa0mhpx3 9 06L...,id 0 01Yfj2T3YTwJ1Yfy 8 06H...,id 6 04iitW3ffa0mhpx3 9 06L...
6,04iitW3ffa0mhpx3,Ne-Yo,Miss Independent,Year Of The Gentleman (Bonus Track Edition),https://www.youtube.com/watch?v=k6M5C-oKw9k,"[pop, r b, hip hop, soul, rhythm and blues, si...",id 1 01gyRHLquwXDlhkO 7 04x...,id 1 01gyRHLquwXDlhkO 3 02R...,id 9 06L9OJ5nRqKnO2q9 5 04O...,id 1 01gyRHLquwXDlhkO 5 04O...,id 4 02ZnlCGZEbkfCDxo 2 01r...,id 5 04OjszRi9rC5BlHC 9 06L...,id 8 06HvNTU9M9lnH71I 1 01g...,id 3 02RGE9FNH65RtMS7 9 06L...
7,04xUDjAYC14jsHyH,Jawbreaker,Jinx Removing,24 Hour Revenge Therapy (Remastered),https://www.youtube.com/watch?v=MLeekX2znNU,"[punk, emo, post hardcore, post punk, melodic ...",id 1 01gyRHLquwXDlhkO 8 06H...,id 4 02ZnlCGZEbkfCDxo 8 06H...,id 9 06L9OJ5nRqKnO2q9 4 02Z...,id 0 01Yfj2T3YTwJ1Yfy 3 02R...,id 0 01Yfj2T3YTwJ1Yfy 2 01r...,id 0 01Yfj2T3YTwJ1Yfy 9 06L...,id 0 01Yfj2T3YTwJ1Yfy 4 02Z...,id 9 06L9OJ5nRqKnO2q9 1 01g...
8,06HvNTU9M9lnH71I,Michael Bublé,That's All,Michael Bublé (US Version),https://www.youtube.com/watch?v=yGw_uBcU4rU,"[jazz, easy listening, swing, smooth jazz, voc...",id 2 01rMxQv6vhyE1oQX 4 02Z...,id 7 04xUDjAYC14jsHyH 3 02R...,id 7 04xUDjAYC14jsHyH 4 02Z...,id 9 06L9OJ5nRqKnO2q9 5 04O...,id 2 01rMxQv6vhyE1oQX 5 04O...,id 9 06L9OJ5nRqKnO2q9 1 01g...,id 1 01gyRHLquwXDlhkO 0 01Y...,id 6 04iitW3ffa0mhpx3 9 06L...
9,06L9OJ5nRqKnO2q9,Rhye,Pleas

In [16]:
# This is Richards json version
df_info = df_info.head(10)
df_richard = df_info.copy()

# if an error occurs: please delete the song "03Oc9WeMEmyLLQbj" ind information.csv
for index, feature_set, sim_func in rs_info:
    df_richard[f'res_songs_{index}'] = df_info.apply(calc_songs, axis=1, args=(df_info, feature_set, sim_func, ["id", "youtube_link", "genre"]))
    print(f"Finished {index} -------------------")

df_richard = df_richard.drop(columns=["youtube_link", "genre"])

df_richard.to_json(orient='records', indent=2, path_or_buf=f'data-richard.json')
df_richard

01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 
Finished word2vec euc-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 
Finished tfidf cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 
Finished bert cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5BlHC 
04iitW3ffa0mhpx3 
04xUDjAYC14jsHyH 
06HvNTU9M9lnH71I 
06L9OJ5nRqKnO2q9 
Finished mfcc_bow cos-sim -------------------
01Yfj2T3YTwJ1Yfy 
01gyRHLquwXDlhkO 
01rMxQv6vhyE1oQX 
02RGE9FNH65RtMS7 
02ZnlCGZEbkfCDxo 
04OjszRi9rC5B

,id,artist,song,album_name,res_songs_word2vec euc-sim,res_songs_tfidf cos-sim,res_songs_bert cos-sim,res_songs_mfcc_bow cos-sim,res_songs_musicnn cos-sim,res_songs_ivec_256 cos-sim,res_songs_logfluc cos-sim,res_songs_resnet cos-sim
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human,id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition),id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones,id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002),id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te,id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
5,04OjszRi9rC5BlHC,Grizzly Bear,Knife,Yellow House,id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
6,04iitW3ffa0mhpx3,Ne-Yo,Miss Independent,Year Of The Gentleman (Bonus Track Edition),id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
7,04xUDjAYC14jsHyH,Jawbreaker,Jinx Removing,24 Hour Revenge Therapy (Remastered),id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
8,06HvNTU9M9lnH71I,Michael Bublé,That's All,Michael Bublé (US Version),id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
9,06L9OJ5nRqKnO2q9,Rhye,Please,Blood Remixed,id ...,id ...,id ...,id ...,id ...,id ...,id ...,id ...
